In [2]:
import pandas as pd

fixed_df = pd.read_csv('data_fixed_types.csv',low_memory=False)

In [3]:
fixed_df.shape

(3617742, 136)

In [4]:
for i in fixed_df.columns:
    print(i)

YEAR
MONTH
FL_DATE
OP_UNIQUE_CARRIER
OP_CARRIER
OP_CARRIER_FL_NUM
ORIGIN_AIRPORT_ID
ORIGIN
ORIGIN_CITY_NAME
ORIGIN_STATE_ABR
ORIGIN_STATE_NM
DEST_AIRPORT_ID
DEST_AIRPORT_SEQ_ID
DEST
DEST_CITY_NAME
DEST_STATE_ABR
DEST_STATE_NM
DEP_TIME
DEP_DELAY
DEP_DELAY_NEW
DEP_DEL15
DEP_DELAY_GROUP
ARR_TIME
ARR_DELAY
ARR_DELAY_NEW
ARR_DEL15
ARR_DELAY_GROUP
CANCELLED
CANCELLATION_CODE
DIVERTED
AIR_TIME
FLIGHTS
DISTANCE
DISTANCE_GROUP
CARRIER_DELAY
WEATHER_DELAY
NAS_DELAY
SECURITY_DELAY
LATE_AIRCRAFT_DELAY
SOURCE_FILE
ORIGIN_LATITUDE_x
ORIGIN_LONGITUDE_x
ORIGIN_AIRPORT_NAME
ORIGIN_CITY
ORIGIN_COUNTRY
ORIGIN_ALTITUDE
ORIGIN_TIMEZONE
ORIGIN_TZ_DATABASE
DEST_LATITUDE_x
DEST_LONGITUDE_x
DEST_AIRPORT_NAME
DEST_CITY
DEST_COUNTRY
DEST_ALTITUDE
DEST_TIMEZONE
DEST_TZ_DATABASE
DEP_DELAY_MISSING
ARR_DELAY_MISSING
DELAY_CAUSE_REPORTED
IS_DELAYED
ORIGIN_CLUSTER_LAT
ORIGIN_CLUSTER_LON
ORIGIN_CLUSTER_ID
DEST_CLUSTER_LAT
DEST_CLUSTER_LON
DEST_CLUSTER_ID
ORIGIN_LATITUDE_y
ORIGIN_LONGITUDE_y
ORIGIN_TEMP_MAX
ORIGIN_TEMP_

In [5]:
print(f"Total flights in dataset: {len(fixed_df):,}")
print(f"Time period covered: {fixed_df['FL_DATE'].min()} to {fixed_df['FL_DATE'].max()}")
print(f"Number of unique carriers: {fixed_df['OP_UNIQUE_CARRIER'].nunique()}")
print(f"Number of origin airports: {fixed_df['ORIGIN'].nunique()}")
print(f"Number of destination airports: {fixed_df['DEST'].nunique()}")

Total flights in dataset: 3,617,742
Time period covered: 2024-07-01 to 2024-12-31
Number of unique carriers: 15
Number of origin airports: 346
Number of destination airports: 346


In [6]:
delay_stats = {
    'Average Departure Delay (minutes)': fixed_df['DEP_DELAY'].mean(),
    'Average Arrival Delay (minutes)': fixed_df['ARR_DELAY'].mean(),
    'Flights Delayed > 15 min (%)': fixed_df['DEP_DEL15'].mean() * 100,
    'Flights Cancelled (%)': fixed_df['CANCELLED'].mean() * 100,
    'Flights Diverted (%)': fixed_df['DIVERTED'].mean() * 100
}

delay_causes = {
    'Carrier Delay (%)': fixed_df['HAS_CARRIER_DELAY'].mean() * 100,
    'Weather Delay (%)': fixed_df['HAS_WEATHER_DELAY'].mean() * 100,
    'NAS Delay (%)': fixed_df['HAS_NAS_DELAY'].mean() * 100,
    'Security Delay (%)': fixed_df['HAS_SECURITY_DELAY'].mean() * 100,
    'Late Aircraft Delay (%)': fixed_df['HAS_LATE_AIRCRAFT_DELAY'].mean() * 100
}

delay_severity = {
    'Carrier Delay Severity (min)': fixed_df['CARRIER_DELAY_SEVERITY'].mean(),
    'Weather Delay Severity (min)': fixed_df['WEATHER_DELAY_SEVERITY'].mean(),
    'NAS Delay Severity (min)': fixed_df['NAS_DELAY_SEVERITY'].mean(),
    'Security Delay Severity (min)': fixed_df['SECURITY_DELAY_SEVERITY'].mean(),
    'Late Aircraft Delay Severity (min)': fixed_df['LATE_AIRCRAFT_DELAY_SEVERITY'].mean()
}

In [12]:
delay_stats_df = pd.DataFrame(list(delay_stats.items()), columns=['Metric', 'Value'])
delay_causes_df = pd.DataFrame(list(delay_causes.items()), columns=['Cause', 'Percentage'])
delay_severity_df = pd.DataFrame(list(delay_severity.items()), columns=['Cause', 'Minutes'])

print("\n=== Delay Summary Statistics ===")
print(delay_stats_df.to_string(index=False, float_format=lambda x: f"{x:.2f}"))

print("\n=== Delay Causes Breakdown ===")
print(delay_causes_df.to_string(index=False, float_format=lambda x: f"{x:.2f}%"))

print("\n=== Delay Severity by Cause ===")
print(delay_severity_df.to_string(index=False, float_format=lambda x: f"{x:.2f}"))


=== Delay Summary Statistics ===
                           Metric  Value
Average Departure Delay (minutes)  11.51
  Average Arrival Delay (minutes)   5.88
     Flights Delayed > 15 min (%)  19.37
            Flights Cancelled (%)   1.32
             Flights Diverted (%)   0.23

=== Delay Causes Breakdown ===
                  Cause  Percentage
      Carrier Delay (%)      10.78%
      Weather Delay (%)       1.09%
          NAS Delay (%)       9.47%
     Security Delay (%)       0.08%
Late Aircraft Delay (%)       9.82%

=== Delay Severity by Cause ===
                             Cause  Minutes
      Carrier Delay Severity (min)     0.19
      Weather Delay Severity (min)     0.02
          NAS Delay Severity (min)     0.15
     Security Delay Severity (min)     0.00
Late Aircraft Delay Severity (min)     0.20


In [7]:
flight_stats = {
    'Average Flight Distance (miles)': fixed_df['DISTANCE'].mean(),
    'Average Air Time (minutes)': fixed_df['AIR_TIME'].mean(),
    'Short Flights (<500 miles) (%)': (fixed_df['DISTANCE'] < 500).mean() * 100,
    'Medium Flights (500-1500 miles) (%)': ((fixed_df['DISTANCE'] >= 500) & (fixed_df['DISTANCE'] < 1500)).mean() * 100,
    'Long Flights (>1500 miles) (%)': (fixed_df['DISTANCE'] >= 1500).mean() * 100
}

time_stats = {
    'Morning Flights (6AM-12PM) (%)': ((fixed_df['DEP_TIME'] >= 600) & (fixed_df['DEP_TIME'] < 1200)).mean() * 100,
    'Afternoon Flights (12PM-6PM) (%)': ((fixed_df['DEP_TIME'] >= 1200) & (fixed_df['DEP_TIME'] < 1800)).mean() * 100,
    'Evening Flights (6PM-12AM) (%)': ((fixed_df['DEP_TIME'] >= 1800) & (fixed_df['DEP_TIME'] < 2400)).mean() * 100,
    'Night Flights (12AM-6AM) (%)': ((fixed_df['DEP_TIME'] < 600) | (fixed_df['DEP_TIME'] >= 2400)).mean() * 100,
    'Weekend Flights (%)': fixed_df['IS_WEEKEND'].mean() * 100
}

In [13]:
print("\n=== Flight Distance and Duration Statistics ===")
for stat, value in flight_stats.items():
    if "Average" in stat:
        print(f"{stat}: {value:.2f}")
    else:
        print(f"{stat}: {value:.2f}%")


=== Flight Distance and Duration Statistics ===
Average Flight Distance (miles): 830.77
Average Air Time (minutes): 112.24
Short Flights (<500 miles) (%): 34.71%
Medium Flights (500-1500 miles) (%): 52.22%
Long Flights (>1500 miles) (%): 13.07%


In [14]:
print("\n=== Flight Distribution by Time ===")
for period, percentage in time_stats.items():
    print(f"{period}: {percentage:.2f}%")


=== Flight Distribution by Time ===
Morning Flights (6AM-12PM) (%): 36.44%
Afternoon Flights (12PM-6PM) (%): 35.00%
Evening Flights (6PM-12AM) (%): 23.00%
Night Flights (12AM-6AM) (%): 5.56%
Weekend Flights (%): 27.74%


In [8]:
weather_stats = {
    'Average Origin Weather Severity': fixed_df['ORIGIN_WEATHER_SEVERITY'].mean(),
    'Average Destination Weather Severity': fixed_df['DEST_WEATHER_SEVERITY'].mean(),
    'Flights with Extreme Weather at Origin (%)': fixed_df['ORIGIN_EXTREME_WEATHER'].mean() * 100,
    'Flights with Extreme Weather at Destination (%)': fixed_df['DEST_EXTREME_WEATHER'].mean() * 100,
    'Average Weather Impact Score': fixed_df['WEATHER_IMPACT_SCORE'].mean()
}

seasonal_stats = fixed_df['SEASON'].value_counts(normalize=True) * 100

holiday_stats = {
    'Holiday Flights (%)': fixed_df['IS_HOLIDAY'].mean() * 100,
    'Peak Holiday Travel Period Flights (%)': fixed_df['HOLIDAY_TRAVEL_PERIOD'].mean() * 100
}

In [15]:
print("\n=== Weather Impact Statistics ===")
for stat, value in weather_stats.items():
    if "Average" in stat:
        print(f"{stat}: {value:.2f}")
    else:
        print(f"{stat}: {value:.2f}%")


=== Weather Impact Statistics ===
Average Origin Weather Severity: 0.94
Average Destination Weather Severity: 0.94
Flights with Extreme Weather at Origin (%): 6.49%
Flights with Extreme Weather at Destination (%): 6.49%
Average Weather Impact Score: 0.12


In [16]:
print("\n=== Seasonal Distribution ===")
for season, percentage in seasonal_stats.items():
    print(f"{season}: {percentage:.2f}%")


=== Seasonal Distribution ===
Fall: 49.02%
Summer: 34.65%
Winter: 16.32%


In [17]:
print("\n=== Holiday Statistics ===")
for stat, percentage in holiday_stats.items():
    print(f"{stat}: {percentage:.2f}%")


=== Holiday Statistics ===
Holiday Flights (%): 17.62%
Peak Holiday Travel Period Flights (%): 30.27%


In [9]:
carrier_performance = fixed_df.groupby('OP_UNIQUE_CARRIER').agg({
    'DEP_DELAY': 'mean',
    'ARR_DELAY': 'mean',
    'DEP_DEL15': 'mean',
    'CANCELLED': 'mean'
}).sort_values('DEP_DEL15', ascending=False)

carrier_performance['DEP_DEL15'] *= 100  # Convert to percentage
carrier_performance['CANCELLED'] *= 100  # Convert to percentage
carrier_performance.columns = ['Avg Departure Delay (min)', 'Avg Arrival Delay (min)', 
                               'Flights Delayed >15min (%)', 'Flights Cancelled (%)']

In [18]:
print("\n=== Carrier Delay Performance ===")
print(carrier_performance.head(10).to_string(float_format=lambda x: f"{x:.2f}"))


=== Carrier Delay Performance ===
                   Avg Departure Delay (min)  Avg Arrival Delay (min)  Flights Delayed >15min (%)  Flights Cancelled (%)
OP_UNIQUE_CARRIER                                                                                                       
F9                                     15.54                    12.40                       25.19                   2.40
B6                                     15.72                     9.40                       24.18                   1.64
AA                                     18.43                    12.43                       23.62                   1.46
NK                                     14.19                     7.41                       23.57                   2.37
G4                                     14.45                    10.68                       21.45                   2.62
WN                                     10.21                     3.14                       21.22                   0.

In [10]:
top_routes = fixed_df.groupby(['ORIGIN', 'DEST']).size().sort_values(ascending=False).head(20)

worst_delay_routes = fixed_df.groupby(['ORIGIN', 'DEST']).agg({
    'DEP_DELAY': 'mean',
    'ARR_DELAY': 'mean',
    'DEP_DEL15': 'mean'
}).sort_values('DEP_DELAY', ascending=False).head(20)

In [20]:
print("\n=== Top 20 Routes with Highest Average Delays ===")
print(worst_delay_routes.head(20).to_string(float_format=lambda x: f"{x:.2f}"))


=== Top 20 Routes with Highest Average Delays ===
             DEP_DELAY  ARR_DELAY  DEP_DEL15
ORIGIN DEST                                 
JFK    LGA      765.00     755.00       1.00
IAD    MSN      309.00     325.00       1.00
SRQ    IAH      253.83     249.67       0.83
AVL    USA      240.00     310.00       1.00
MVY    CLT      202.67     211.44       0.44
OKC    CAE      174.00     171.00       1.00
LAX    ATW      168.50     164.00       0.50
SAV    BLV      168.00     164.33       0.50
ATW    LAX      165.50     157.00       0.50
BLV    SAV      155.67     159.00       0.50
EGE    JFK      150.27     143.64       0.82
FSD    FLL      143.50     141.25       0.75
SNA    MIA      137.12     119.65       0.29
RAP    XWA      121.00       0.00       1.00
SRQ    DTW      120.91     127.73       0.45
EGE    MIA      117.71     122.04       0.54
HOU    DSM      116.08     118.50       0.50
PGD    SAV      102.71      93.29       0.41
HSV    OKC      102.00      89.50       0.50
RDM 

In [19]:
print("\n=== Top 20 Most Congested Routes ===")
print(top_routes.to_frame('Flight Count').head(20).to_string())


=== Top 20 Most Congested Routes ===
             Flight Count
ORIGIN DEST              
LAX    SFO           6044
SFO    LAX           6039
HNL    OGG           5815
OGG    HNL           5813
LGA    ORD           5605
ORD    LGA           5604
JFK    LAX           5227
LAX    JFK           5222
DCA    BOS           4912
BOS    DCA           4911
LAS    LAX           4905
LAX    LAS           4905
DEN    PHX           4636
PHX    DEN           4633
LIH    HNL           4323
HNL    LIH           4322
MCO    ATL           4294
ATL    MCO           4293
DEN    LAX           4041
LAX    DEN           4040


In [11]:
missing_values = fixed_df.isnull().sum().sort_values(ascending=False)
missing_percent = (missing_values / len(fixed_df)) * 100
missing_summary = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_percent
}).query('`Missing Values` > 0')

In [21]:
print("\n=== Missing Value Analysis ===")
print(missing_summary.to_string(float_format=lambda x: f"{x:.2f}%"))


=== Missing Value Analysis ===
Empty DataFrame
Columns: [Missing Values, Percentage (%)]
Index: []
